## For version 2.0.0
#### 8x8 systolic array
#### Under the system's capability (one matrix size under XXX), you can run large matrix muliplication on FPGA  

In [4]:
from pynq import Overlay
ol = Overlay("version_1.9.00.bit")

In [5]:
ol?

In [6]:
import pynq
# You can change the clock frequency #
pynq.ps.Clocks.fclk0_mhz = 10 #(unit: MHz)
print(pynq.ps.Clocks.fclk0_mhz)

10.0


#### This is the instance of bram controller, you can insert the values via using this.

In [7]:
bram_sp_arr = ol.axi_bram_ctrl_0.mmio.array
bram_a_arr = ol.axi_bram_ctrl_1.mmio.array
bram_w_arr = ol.axi_bram_ctrl_2.mmio.array
bram_o_arr = ol.axi_bram_ctrl_3.mmio.array

#### The matmul tile input must be padded to satisfy 8 width.
#### bram_sp_arr[0:11] = [0,1,M,K,N, num_iter_row, num_iter_col,num_iter_K , step_a, step_w, step_o]

In [73]:
import numpy as np
import random 
import time

def matmul_OS_proj3  (mat1: np.array, mat2: np.array):
    dim = 8
    assert(mat1.shape[1] == mat2.shape[0])
    M = mat1.shape[0]
    K = mat1.shape[1]
    N = mat2.shape[1]
    num_iter_row = int(M/dim) +1 - int(M%dim == 0)
    num_iter_col = int(N/dim) +1 - int(N%dim == 0) 
    num_iter_K = 1
    step_a = 8    # notuse
    step_w =  8   # notuse
    step_o = 8    # notuse
    
    # padding mat1 and mat2, so that mat1.shape[0] and mat2.shape[1] be 8
    # PADDING
    if(M%dim == 0): padding_M = 0
    else: padding_M = dim - M%dim
    if(N%dim == 0): padding_N = 0
    else: padding_N = dim - N%dim
    # ======insert tiles into BRAM ===== #
    for row in range (num_iter_row):
        if(row < num_iter_row -1): mat1_tile = mat1[dim*row:dim*(row+1) , :]
        else: mat1_tile = np.pad(mat1[dim*row: , :], ((0,padding_M),(0,0)), 'constant', constant_values =0)
        assert(mat1_tile.shape[0] == 8)
        # serialize the matrix upload to BRAM (mat1: row wise)
        serial_mat1 = np.reshape(mat1_tile, -1, order = 'F')
        # load data to BRAM 
        bram_a_arr[row*dim*K:(row+1)*dim*K] = serial_mat1   
    for col in range (num_iter_col):
        if(col < num_iter_col -1): mat2_tile = mat2[ : ,col*dim : (col+1)*dim]
        else: mat2_tile = np.pad(mat2[ : ,col*dim : ],((0,0),(0,padding_N)), 'constant', constant_values =0)   
        assert(mat2_tile.shape[1] == 8)
        # serialize the matrix upload to BRAM (mat1: row wise | mat2: col wise)
        serial_mat2 = mat2_tile.flatten()
        # load data to BRAM 
        bram_w_arr[col*dim*K:(col+1)*dim*K] = serial_mat2
    
    #=============== HW RUNNIG ==================#
    # set Special Memory to specify M,K,N and mode (1: OS)    
    bram_sp_arr[0:11] = [0,1,M,K,N, num_iter_row, num_iter_col,num_iter_K , step_a, step_w, step_o]
    # start (set sp(addr0) = 1)
    bram_sp_arr[0] = 1
    #while until sp(addr0) => 0 or sp(addr100) =>1 
    while(bram_sp_arr[25] != 1):
        pass
    # get data from BARM_O
    reshape_fit = np.reshape(bram_o_arr[0:dim*dim*num_iter_row*num_iter_col], (-1,8))
    #=============== HW END =====================#
    
    # Re-locate the tiles 
    reshape_fit = reshape_fit[::-1]
    base = num_iter_col * num_iter_row - 1
    for col in range (num_iter_col):
        for row in range (num_iter_row):
            matmul_result = reshape_fit[dim*(base - (col*num_iter_row+row)):dim*(base - (col*num_iter_row+row-1)), :]
            if(col != num_iter_col-1 and row != num_iter_row-1):
                matmul_result = matmul_result
            elif(col != num_iter_col-1 and row == num_iter_row-1):
                matmul_result = matmul_result[padding_M: , :]
            elif(col == num_iter_col-1 and row != num_iter_row-1):
                matmul_result = matmul_result[: , :dim-padding_N]
            else:
                matmul_result = matmul_result[padding_M: , :dim-padding_N]
            # concat row_direction
            if(row == 0):
                concat_row_dir = matmul_result
                time.sleep(0.001)
            else:
                concat_row_dir = np.concatenate((concat_row_dir,matmul_result), axis = 0)
#                 time.sleep(0.001)
        # concat col_direction
        if(col == 0):
            concat_col_dir = concat_row_dir
            time.sleep(0.001)
        else:
            concat_col_dir = np.concatenate((concat_col_dir,concat_row_dir), axis = 1)
#             time.sleep(0.001)
            

    return concat_col_dir.astype(np.int32)

def matmul_WS_proj3 (mat1: np.array, mat2: np.array):
    assert(mat1.shape[1]==mat2.shape[0])
    dim = 8
    depth = 8
    # SP_BRAM parameters
    M = mat2.shape[0]
    K = mat1.shape[0]
    N = mat2.shape[1]
    num_iter_row = int(M/dim) +1 - int(M%dim == 0)
    num_iter_col = int(N/dim) +1 - int(N%dim == 0) 
    num_iter_K = int(K/depth) +1 -int(K%depth == 0)
    step_a = 8
    step_w =  8  
    step_o = 8

    # PADDING
    if(K%dim == 0): padding_K = 0
    else: padding_K = dim - K%dim
    if(M%dim == 0): padding_M = 0
    else: padding_M = dim - M%dim
    if(N%dim == 0): padding_N = 0
    else: padding_N = dim - N%dim
        
    # =====insert tiles into BRAM  ==== #   
    for k in range (num_iter_K):
        for i in range (num_iter_row):
            if(i < num_iter_row -1 and k < num_iter_K -1):
                tile_mat1 = mat1[dim*k:dim*(k+1), dim*i:dim*(i+1)]
            elif(i == num_iter_row -1 and k < num_iter_K -1): 
                tile_mat1 = np.pad(mat1[dim*k:dim*(k+1), dim*i:],((0,0),(0,padding_M)))
            elif(i < num_iter_row -1 and k == num_iter_K -1): 
                tile_mat1 = np.pad(mat1[dim*k:, dim*i:dim*(i+1)],((0,padding_K),(0,0)))
            else:
                tile_mat1 = np.pad(mat1[dim*k:, dim*i:],((0,padding_K),(0,padding_M)))

            serial_tile_mat1 = tile_mat1.flatten()
            bram_a_arr[dim*dim*(i+k*num_iter_row):dim*dim*(i+k*num_iter_row+1)] = serial_tile_mat1
    
    for j in range (num_iter_col):
        for i in range (num_iter_row):
            if(i < num_iter_row -1 and j < num_iter_col -1):
                tile_mat2 = mat2[dim*i:dim*(i+1),dim*j:dim*(j+1)]
                tile_mat2 = tile_mat2[::-1]
                tile_mat2 = np.pad(tile_mat2, ((0,0),(0,0)))
            elif(i < num_iter_row -1 and j == num_iter_col -1):
                tile_mat2 = mat2[dim*i:dim*(i+1),dim*j:]
                tile_mat2 = tile_mat2[::-1]
                tile_mat2 = np.pad(tile_mat2, ((0,0),(0,padding_N)))
            elif(i == num_iter_row -1 and j < num_iter_col -1):
                tile_mat2 = mat2[dim*i:,dim*j:dim*(j+1)]
                tile_mat2 = tile_mat2[::-1]
                tile_mat2 = np.pad(tile_mat2, ((0,padding_M),(0,0)))
            else:
                tile_mat2 = mat2[dim*i:,dim*j:]
                tile_mat2 = tile_mat2[::-1]
                tile_mat2 = np.pad(tile_mat2, ((0,padding_M),(0,padding_N)))

            serial_tile_mat2 = tile_mat2.flatten()
            bram_w_arr[dim*dim*(i+j*num_iter_row):dim*dim*(i+j*num_iter_row+1)] = serial_tile_mat2
    #=============== HW RUNNIG ==================#
    # set Special Memory to specify M,K,N and mode (1: OS)
    bram_sp_arr[0:11] = [0,0,M,K,N, num_iter_row, num_iter_col,num_iter_K , step_a, step_w, step_o]
                        #x,0,1,2,3,4            ,5            ,6          ,7
    # start (set sp(addr0) = 1)
    bram_sp_arr[0] = 1
    # while until sp(addr0) => 0 or sp(addr100) =>1 
    while(bram_sp_arr[25] != 1):
        pass
    #=============== HW END =====================#
    # re-locate the tiles 
    for n in range (num_iter_col):
        if(n == 0):
            matmul_result = np.reshape(bram_o_arr[dim*depth*num_iter_K*n : dim*depth*num_iter_K*(n+1)], (-1,dim))
            time.sleep(0.001)
        else:
            matmul_result = np.concatenate((matmul_result, np.reshape(bram_o_arr[dim*depth*num_iter_K*n : dim*depth*num_iter_K*(n+1)], (-1,dim))), axis = 1)

    return matmul_result.astype(np.int32)[:K,:N]

### The maximum matrix size depends on BRAM size of MODULE. (TODO)
### The range of values in the matrix depend on size of K (OS) and size of M (WS) [output is INT32]
#### 10000번에 1 2번 맨 앞 2개 값만 다르게 나옴 (max 32)
#### WS도 비슷함 증상이
#### 둘다 확실한 안정 범위는 (M,N,K < 24)

In [89]:
import random

def test_OS():
    score = 0
    num_test = 0
    ## OUTPUT STATIONARY TEST
    for _ in range (1000):
        M = random.randint(1,40)
        K = random.randint(1,51)
        N = random.randint(1,40)
        num_test = num_test +1
        mat1 = np.random.randint(-(1<<12), 1<<12 ,size=(M,K)) 
        mat2 = np.random.randint(-(1<<12), 1<<12 ,size=(K,N)) 
#         print("M: "+str(M) +" N:  "+str(K) +" K: "+str(N) +" ")
        CPU = mat1@mat2
        FPGA = matmul_OS_proj3(mat1, mat2)

        if(np.equal(CPU,FPGA).all()):
            score = score +1
        else:
            print("M: "+str(M) +" N:  "+str(K) +" K: "+str(N) +" ")
            print(CPU)
            print(FPGA)
            print(np.equal(FPGA, CPU))
            
    return score/num_test * 100


def test_WS():
    score = 0
    num_test = 0
    ## WEIGHT STATIONARY TEST
    for iter_ in range (100):
        
        M = random.randint(1,48)
        K = random.randint(1,40)
        N = random.randint(1,40)
        num_test = num_test +1
        mat1 = np.random.randint(-10,10,size=(K,M)) #np.ones((K,M))#
        mat2 = np.random.randint(-10, 10 ,size=(M,N))#np.ones((M,N))#
#         print("M: "+str(M) +" N:  "+str(K) +" K: "+str(N) +" ")
        FPGA = matmul_WS_proj3(mat1, mat2)
        CPU = mat1@mat2
        if(np.equal(FPGA, CPU).all()):
            score = score + 1
        else:
            print("M: "+str(M) +" N:  "+str(K) +" K: "+str(N) +" ")
            print(CPU)
            print(FPGA)
            print(np.equal(FPGA, CPU))
    return score/num_test*100

In [86]:
test_OS()

100.0

In [90]:
test_WS()

100.0

In [77]:
def project3_test():
    print(" SCORE : " + str((test_WS() + test_OS()) * 0.5) + " / 100 " )

In [78]:
project3_test()

 SCORE : 100.0 / 100 


In [79]:
def probe_time():
    import time
    M = 24
    K = 24
    N = 24
    mat1 = np.random.randint(-(1<<15),1<<15 -1 ,size=(K,M)) 
    mat2 = np.random.randint(-(1<<15),1<<15 -1 ,size=(M,N)) 
    start_fpga_WS = time.time()
    matmul_WS_proj3(mat1, mat2)
    end_fpga_WS = time.time()
    print(end_fpga_WS - start_fpga_WS)
    
    start_fpga_OS = time.time()
    matmul_OS_proj3(mat1, mat2)
    end_fpga_OS = time.time()
    print(end_fpga_OS - start_fpga_OS)
    
    start_python = time.time()
    mat1@mat2
    end_python = time.time()
    print(end_python - start_python)
    print("ratio_WS   " + str((end_fpga_WS - start_fpga_WS)/(end_python - start_python)))
    print("ratio_OS   " + str((end_fpga_OS - start_fpga_OS)/(end_python - start_python)))

In [80]:
probe_time()

0.024097442626953125
0.010792255401611328
0.00021147727966308594
ratio_WS   113.94813979706878
ratio_OS   51.03269447576099
